#Overview of Assignment:
The purpose of this Simulation is to simulate a population of animals and who can do basic interaction with eachother and others and to be able to use this simulation to run tests to find the best way to maximise how long they will last with diffirent combinations of numbers of animals and food.

My take on the PaSciRo simulation slightly varied from some suggestions provided in the assignment description. I used three animals but I didn't have have a paper, scissors, rock way of describing which animal would eat which, I decided to structure it around real life food chains, food eaten by omnivore and herbivore and those two types of animals eaten by the Carnivore. 

To try and balance these animals, I had created unique movement patterns with the thought that omnivores, as they are allowed to eat both food and herbivores, had the worst movement (up, down left, right). Herbivores had a slighly better movement pattern being able to also move in the diagonal directions. Carnivores were given the most mobile movement of the animals, since they required eating moving animals, I gave them 2 tile movement steps.

I had originally wanted to have the animals find the closest animal/food when they were hungry and then pathfind their way towards that target objects, from past experience with pathfinding algorithms, I was considering implementing the A* pathfinding algorithm but due to time constraints (creating documentation), I decided that it wasn't worth it.

The food was meant to be a basic simulation of how plants grow, each turn they would increase in hunger replenishement value (by 1) and when they got past a certain value, they had a chance to spread to a random tile around them. This meant that with the right parameter values, the animals could live continuosly together without making any animal go extinct.

The way I created these animals and their movement was through classes. I had a master class for animals which had the __init__ function creating all of the variables and functions which were commmon to all animals, then I created a sub class for each of the separate animals which had a movment function which also encompassed moving.
I originally thought that I would use more functions for the subclasses but turned out that I only needed one.

In [ ]:
#This is the master class for all animals
class animal_master():
    def __init__(self, position, hunger, subtype):
        self.current = position
        self.hunger = hunger
        self.type = 'animal'
        self.subtype = subtype
        self.turn = 0
        self.t_since_last_xxx = 1
    def update_self(self, location_array):
        '''
        updates timestep based variables like time_since_last_xxx (last procreation) and hunger
        '''
        if self.t_since_last_xxx != 0:
            self.t_since_last_xxx += 1
        self.hunger -= 1
        if self.hunger == 0:
            location_array[self.current[0]][self.current[1]] = None
    def procreate(self, location_array, chance_xxx): #can only procreate once per x turns
        nearbye = []
        empty = []

        #splits and converts the current location of the object to shorter and easier to type variables
        x = self.current[1]
        y = self.current[0]

        #Checks to see if hunger is greater than a certain point and that it hasn't procreated in a certain time frame
        if self.hunger >= 5 and self.t_since_last_xxx >= 5:
            #iterates through the location array and appends empty tiles around the objects current location
            for j in range(y-1, y+2):
                for i in range(x-1, x+2):
                    if [j, i] == [y, x]:
                        pass
                    elif i in range(0, len(location_array[0])) and j in range(0, len(location_array)):
                        if location_array[j][i] is None:
                            empty.append([j, i])
                        elif location_array[j][i].type == 'animal':
                            if location_array[j][i].subtype == self.subtype and location_array[j][i].hunger >= 5:
                                nearbye.append([j, i])


            '''checks chance to procreate and then checks if required parameters are met(nearbye animals of same type
            and empty tiles nearbye)
            '''
            if len(nearbye) >= 1 and random.randint(0,100) <= chance_xxx and len(empty) > 0:
                choice_xxx = random.choice(nearbye)
                location_array[choice_xxx[0]][choice_xxx[1]].t_since_last_xxx = 0
                location_array[choice_xxx[0]][choice_xxx[1]].hunger -= 3
                self.hunger -= 3
                self.t_since_last_xxx = 0
                '''updates hunger of parent animals(when procreate, subtracts some hunger) and also resets the counter 
                for time_since_last_xxx
                '''
                location_baby = random.choice(empty)
                #Checks which type of animal is procreating and then creates a 'baby' of that type near the parent
                if self.subtype == 'animal_1':
                    location_array[location_baby[0]][location_baby[1]] = animal_1(location_baby, 5, 'animal_1')
                    location_array[location_baby[0]][location_baby[1]].turn = self.turn
                if self.subtype == 'animal_2':
                    location_array[location_baby[0]][location_baby[1]] = animal_2(location_baby, 5, 'animal_2')
                    location_array[location_baby[0]][location_baby[1]].turn = self.turn
                if self.subtype == 'animal_3':
                    location_array[location_baby[0]][location_baby[1]] = animal_3(location_baby, 5, 'animal_3')
                    location_array[location_baby[0]][location_baby[1]].turn = self.turn

In [ ]:
#This is the subclass for animal 1
class animal_1(animal_master):
    # herbivore animal (only eats 'food')
    def movement(self, location_array, when_eat):
        '''
        Movement of animal_1 encompasses both moving and eating
        Priorities of moving and eating are as follow:
        1. If hunger is below a certain threshold, look for and eat food
        2. Move to a random tile in a 1 block radius around its current position
        '''

        # splits object's current location to easier to type variables
        x = self.current[1]
        y = self.current[0]
        movement = []

        # adds tiles in a 1 block radius around the current location to a list (also checking if in bounds of 'arena'
        for j in range(y-1, y+2):
            for i in range(x-1, x+2):
                if [j,i] == [y, x]:
                    pass
                elif i in range(0, len(location_array[0])) and j in range(0, len(location_array)):
                    movement.append([j, i])

        choices = []
        eat_tile = []
        move = []
        # checks to see what is in the tiles nearbye and takes note of where the food and empty tiles are
        for points in movement:
            if location_array[points[0]][points[1]] is None:
                choices.append(points)
            elif location_array[points[0]][points[1]].subtype == 'food':
                eat_tile.append(points)

        if len(eat_tile) >= 1 and self.hunger <= when_eat:
            # Animal 1 has a long reach therefore, when eating food, can eat adjacent tile without moving to it
            move = random.choice(eat_tile)
            self.hunger += location_array[move[0]][move[1]].value
            location_array[move[0]][move[1]] = None
        elif len(choices) > 0:
            # checks to see if there are empty tiles otherwise doesn't move anywhere
            move = random.choice(choices)
            location_array[y][x], location_array[move[0]][move[1]] = location_array[move[0]][move[1]],  location_array[y][x]

        if len(move) == 0:
            self.turn += 1
        else:
            self.current = move
            self.turn += 1
        return location_array

For food, I didn't need any subclasses, just a single class which outlined the basic things that it does i.e (spreading and updating self)

To smoothen out the creation stage of all the animals, I created functions to loop create individual animals with unique locations in empty spaces. This was done for both animals and food

In [ ]:
def gencoordinates(fieldsize): #generating a random x and y value between 0 and the max x and y fieldsize
    x = random.randint(0, fieldsize[0]-1)
    y = random.randint(0, fieldsize[1]-1)
    return x, y
def genfood(n_food, location_array, fieldsize): #generates the location of food for omnivores and herbivores
    c = 0
    #making sure that all food is generated
    while c != n_food:
        x, y = gencoordinates(fieldsize)
        #checking to that the x y coordinates in the location array is empty
        if location_array[y][x] is None:
            location = [y, x]
            #Generating food at empty location
            location_array[y][x] = food(location)
            c += 1

To smooth out iterating through all objects, I created a 2 dimensional numpy array of the objects which meant that my program just had to do a double for loop to look at the objects. The only problem that arose from this was that some objects were moving to parts of the array which were not itterated on (therefore moving multiple times in 1 turn), this lead to me needing to add a move counter to make sure that all objects only had 1 turn per cycle.

I also created some other functions to make the test code shorter and easier to write. This included, converting the object array to a number array for saving to csv files, a function to save to csv files, animal counter and pygame display function.

#How to use Assignment_Test.py
Only you need to worry about is the variables under the commend "Changeable variables"
All of these variable can be changed and edited and will provide a different result.

NOTE: The test file is designed to run until there are no animals left on the field or when the max turn counter is met

In [1]:
fieldsize = [20,20]
n_food = 35 #plants (Green)
n_animal_1 = 15 #Herbivore (Yellow)
n_animal_2 = 7 #Carnivore (Orange)
n_animal_3 = 12 #Omnivore (Magenta)
chance_xxx = 100
chance_spread = 15
starting_hunger = random.randint(10, 15)
when_eat = 10
width, height = 750, 750
filename = open('test.csv','w')
max_it = 30

NameError: name 'random' is not defined

- Field: [x,y] is how many blocks wide ('x') the object array is and how many blocks long it is ('y'). This is important as the objects cannot move out of this field, so if it is too cramped, most of them won't be able to move

- n_food: the number of the food objects to be created at start of simulation

- n_animal_1: the number of animal of type 1 to be created at start of simulation

- n_animal_2: the number of animal of type 2 to be created at start of simulation

- n_animal_3: the number of animal of type 1 to be created at start of simulation

- chance_xxx: chance for an animals of the same type meeting all other condition to procreate with one another

- chance_spread: chance for the food tile to spread to an adjacent tile given that other condition are met

- starting_hunger (x,y): the starting hunger level for all animals created, is a random integer between x and y

- when_eat : The hunger level at which the animals will eat at

- width, height = x, y : Size of the pygame window that displays graphical representation of the location of all animals

- filename: the name of the file which the data created from the simulation will be saved to.

- max_it: The maximum ammount of turns which will be completed in the simulation (to stop infinite simulation and oversized data files)

#How to use Assignment_Test_argv.py

It is the exact same as Assignment_test.py except, you call in the variables outisde of the code.

It is in the exact same order as the variables are listed above

In [ ]:
fieldsize = [sys.argv[1],sys.argv[2]]
n_food = sys.argv[3] #plants (Green)
n_animal_1 = sys.argv[4] #Herbivore (Yellow)
n_animal_2 = sys.argv[5] #Carnivore (Orange)
n_animal_3 = sys.argv[6] #Omnivore (Magenta)
chance_xxx = sys.argv[7]
chance_spread = sys.argv[8]
starting_hunger = random.randint(sys.argv[9], sys.argv[10])
when_eat = sys.argv[11]
width, height = sys.argv[13], sys.argv[14]
filename = open(sys.argv[15],'w')
max_it = sys.argv[16]

#How to use Assignment_test_counter.py

This code only saves the counter to a file and you can also have multiple iterations of it. (also added a directory to create/save the file to)

In [ ]:
fieldsize = [25,25]
n_food = 50
chance_xxx = 20 #chance for reproduction
chance_spread = 5 #chance for food to spread
starting_hunger = random.randint(12, 18) #starting hunger value for animals
when_eat = 10 #when the animals prioritise eating
max_it = 500
max_animal_total = 45
repeat_x = 15
filename = 'test.csv'
directory = r'C:\Users\diamo\OneDrive\Desktop\Programming Assignment\Simulation_Results'

#How to use Assignment_test_counter_arg.py

just like Assignment_test_counter.py although you need to parse the variables from the command line.

In [ ]:
fieldsize = [sys.argv[1],sys.argv[2]]
n_food = sys.argv[3] #plants (Green)
chance_xxx = sys.argv[4]
chance_spread = sys.argv[5]
starting_hunger = random.randint(sys.argv[6], sys.argv[7])
when_eat = sys.argv[8]
max_it = sys.argv[9]
max_animal_total = sys.argv[10]
directory = sys.argv[13]
filename = sys.argv[12]
repeat_x = sys.argv[11]

#How to use Assignment_Test_movesave.py

This code saves all the moves to a file.
Variables are pretty similar

In [ ]:
fieldsize = [20,20]
n_food = 35 #plants (Green)
chance_xxx = 20
chance_spread = 5
starting_hunger = random.randint(10, 15)
when_eat = 10
width, height = 750, 750
max_it = 100
max_animal_total = 50
directory =  r'C:\Users\diamo\OneDrive\Desktop\Programming Assignment\Simulation_Results'
file_start_name = 'test.csv'

#How to use Assignment_Test_movesave_arg.py

This is same as Assignment_Test_movesave.py except you parse in the variables from the command line.

In [ ]:
fieldsize = [sys.argv[1],sys.argv[2]]
n_food = sys.argv[3] #plants (Green)
chance_xxx = sys.argv[4]
chance_spread = sys.argv[5]
starting_hunger = random.randint(sys.argv[6], sys.argv[7])
when_eat = sys.argv[8]
max_it = sys.argv[9]
max_animal_total = sys.argv[10]
directory =  sys.argv[11]
file_start_name = sys.argv[12]

#References
Khushlani, L., 2020. What Is The Surface.Blit() Function In Pygame? What Does It Do? How Does It Work?. [online] Stack Overflow. Available at: <https://stackoverflow.com/questions/37800894/what-is-the-surface-blit-function-in-pygame-what-does-it-do-how-does-it-work> [Accessed 21 October 2020].

Programiz.com. 2020. Writing CSV Files In Python. [online] Available at: <https://www.programiz.com/python-programming/writing-csv-files> [Accessed 24 October 2020].

Codegrepper.com. 2020. Average Out All Rows Pandas Code Example. [online] Available at: <https://www.codegrepper.com/code-examples/python/average+out+all+rows+pandas> [Accessed 28 October 2020].

Stack Overflow. 2017. Add A Row With Means Of Columns To Pandas Dataframe. [online] Available at: <https://stackoverflow.com/questions/42865805/add-a-row-with-means-of-columns-to-pandas-dataframe> [Accessed 29 October 2020].